In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy.stats import pearsonr, ttest_ind

In [2]:
df = pd.read_csv("QVI_data.csv")

In [3]:
df.head(2)

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream


In [4]:
# Convert DATE column to datetime
df['DATE'] = pd.to_datetime(df['DATE'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   LYLTY_CARD_NBR    264834 non-null  int64         
 1   DATE              264834 non-null  datetime64[ns]
 2   STORE_NBR         264834 non-null  int64         
 3   TXN_ID            264834 non-null  int64         
 4   PROD_NBR          264834 non-null  int64         
 5   PROD_NAME         264834 non-null  object        
 6   PROD_QTY          264834 non-null  int64         
 7   TOT_SALES         264834 non-null  float64       
 8   PACK_SIZE         264834 non-null  int64         
 9   BRAND             264834 non-null  object        
 10  LIFESTAGE         264834 non-null  object        
 11  PREMIUM_CUSTOMER  264834 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(6), object(4)
memory usage: 24.2+ MB


In [6]:
#A function to calculate the monthly metrics
def calculate_monthly_metrics(data):
    return (
        data.groupby(['STORE_NBR', pd.Grouper(key='DATE', freq='ME')])
        .agg(
            total_sales=('TOT_SALES', 'sum'),
            total_customers=('LYLTY_CARD_NBR', 'nunique'),
            avg_transactions=('PROD_QTY', lambda x: x.sum() / x.nunique())
        )
        .reset_index()
    )

In [7]:
# Calculate monthly metrics
monthly_metrics = calculate_monthly_metrics(df)
print(monthly_metrics)

      STORE_NBR       DATE  total_sales  total_customers  avg_transactions
0             1 2018-07-31        206.9               49              31.0
1             1 2018-08-31        176.1               42              18.0
2             1 2018-09-30        278.8               59              37.5
3             1 2018-10-31        188.1               44              29.0
4             1 2018-11-30        192.6               46              28.5
...         ...        ...          ...              ...               ...
3164        272 2019-02-28        395.5               45              45.5
3165        272 2019-03-31        442.3               50              50.5
3166        272 2019-04-30        445.1               54              52.5
3167        272 2019-05-31        314.6               34              35.5
3168        272 2019-06-30        312.1               34              35.0

[3169 rows x 5 columns]


In [8]:
# Function to calculate similarity
def calculate_similarity(trial_store, control_stores, metric='pearson'):
    similarities = []
    for control_store in control_stores:
        trial_data = trial_store[['DATE', 'total_sales']].set_index('DATE')
        control_data = control_store[['DATE', 'total_sales']].set_index('DATE')
        
        # Match dates
        merged_data = trial_data.join(control_data, lsuffix='_trial', rsuffix='_control').dropna()
        
        # Pearson correlation
        if metric == 'pearson':
            corr, _ = pearsonr(merged_data['total_sales_trial'], merged_data['total_sales_control'])
            similarities.append((control_store['STORE_NBR'].iloc[0], corr))
        elif metric == 'magnitude':
            distance = np.abs(
                merged_data['total_sales_trial'] - merged_data['total_sales_control']
            ).mean()
            similarities.append((control_store['STORE_NBR'].iloc[0], 1 - distance))
    
    # Return the best match
    return max(similarities, key=lambda x: x[1])


In [9]:
# Separate trial and control stores
trial_stores = [77, 86, 88]
control_stores = monthly_metrics[~monthly_metrics['STORE_NBR'].isin(trial_stores)]

In [13]:
from scipy.stats import pearsonr

best_control_stores = {}

# Loop through each trial store
for trial_store_num in trial_stores:
    trial_store_data = monthly_metrics[monthly_metrics['STORE_NBR'] == trial_store_num]
    print(f"Processing Trial Store {trial_store_num}...")
    
    # Check if trial_store_data has enough data
    if trial_store_data.empty:
        print(f"No data for Trial Store {trial_store_num}. Skipping.")
        continue

    # Initialize a list to store similarities for the current trial store
    similarities = []

    # Iterate over each potential control store
    for store_num in control_stores['STORE_NBR'].unique():
        control_data = control_stores[control_stores['STORE_NBR'] == store_num]

        # Merge trial and control data on the date column
        merged_data = trial_store_data.join(
            control_data.set_index('DATE'), 
            on='DATE', 
            lsuffix='_trial', 
            rsuffix='_control'
        ).dropna()

        # Check if there are at least 2 rows for Pearson correlation
        if len(merged_data) < 2:
            print(f"Insufficient data for Control Store {store_num}. Skipping.")
            continue

        # Calculate similarity using Pearson correlation
        try:
            corr, _ = pearsonr(merged_data['total_sales_trial'], merged_data['total_sales_control'])
            similarities.append((store_num, corr))
        except Exception as e:
            print(f"Error calculating similarity for Control Store {store_num}: {e}")

    # Select the control store with the highest similarity
    if similarities:
        best_control_store = max(similarities, key=lambda x: x[1])
        best_control_stores[trial_store_num] = best_control_store
    else:
        print(f"No suitable control stores found for Trial Store {trial_store_num}.")

print("Best control stores for each trial store:", best_control_stores)


Processing Trial Store 77...
Insufficient data for Control Store 76. Skipping.
Insufficient data for Control Store 85. Skipping.
Insufficient data for Control Store 92. Skipping.


C:\Users\user\AppData\Local\Temp\ipykernel_17360\4237740904.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(merged_data['total_sales_trial'], merged_data['total_sales_control'])
C:\Users\user\AppData\Local\Temp\ipykernel_17360\4237740904.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(merged_data['total_sales_trial'], merged_data['total_sales_control'])


Processing Trial Store 86...
Insufficient data for Control Store 76. Skipping.
Insufficient data for Control Store 85. Skipping.
Insufficient data for Control Store 92. Skipping.


C:\Users\user\AppData\Local\Temp\ipykernel_17360\4237740904.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(merged_data['total_sales_trial'], merged_data['total_sales_control'])
C:\Users\user\AppData\Local\Temp\ipykernel_17360\4237740904.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(merged_data['total_sales_trial'], merged_data['total_sales_control'])


Processing Trial Store 88...
Insufficient data for Control Store 76. Skipping.
Insufficient data for Control Store 85. Skipping.
Insufficient data for Control Store 92. Skipping.


C:\Users\user\AppData\Local\Temp\ipykernel_17360\4237740904.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(merged_data['total_sales_trial'], merged_data['total_sales_control'])


Best control stores for each trial store: {77: (11, 1.0), 86: (31, 1.0), 88: (206, 1.0)}


C:\Users\user\AppData\Local\Temp\ipykernel_17360\4237740904.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(merged_data['total_sales_trial'], merged_data['total_sales_control'])


In [10]:
# Function to compare trial and control store performance
def compare_trial_control(trial_data, control_data, trial_period):
    trial = trial_data[trial_data['DATE'].between(trial_period[0], trial_period[1])]
    control = control_data[control_data['DATE'].between(trial_period[0], trial_period[1])]
    
    # Total sales
    t_stat, p_value = ttest_ind(trial['total_sales'], control['total_sales'], equal_var=False)
    
    # Breakdown
    customer_change = trial['total_customers'].sum() - control['total_customers'].sum()
    transaction_change = trial['avg_transactions'].mean() - control['avg_transactions'].mean()
    
    return {
        'p_value': p_value,
        'customer_change': customer_change,
        'transaction_change': transaction_change,
    }

In [15]:
# Define the trial period
trial_period = ('2019-01-01', '2019-03-31')

# Compare each trial and control store
for trial_store_num, control_store_info in best_control_stores.items():
    control_store_num = control_store_info[0]
    
    # Get trial and control data
    trial_store_data = monthly_metrics[
        (monthly_metrics['STORE_NBR'] == trial_store_num) &
        (monthly_metrics['DATE'] >= trial_period[0]) &
        (monthly_metrics['DATE'] <= trial_period[1])
    ]
    control_store_data = monthly_metrics[
        (monthly_metrics['STORE_NBR'] == control_store_num) &
        (monthly_metrics['DATE'] >= trial_period[0]) &
        (monthly_metrics['DATE'] <= trial_period[1])
    ]
    
    # Check if data is sufficient
    if trial_store_data.empty or control_store_data.empty:
        print(f"Insufficient data for Trial Store {trial_store_num} or Control Store {control_store_num}. Skipping.")
        continue

    # Compare performance
    results = compare_trial_control(trial_store_data, control_store_data, trial_period)

    # Debugging outputs
    print(f"Trial Store {trial_store_num} Data:\n", trial_store_data)
    print(f"Control Store {control_store_num} Data:\n", control_store_data)
    print(f"Results for Trial Store {trial_store_num} vs Control Store {control_store_num}:")
    print(results)


Insufficient data for Trial Store 77 or Control Store 11. Skipping.
Insufficient data for Trial Store 86 or Control Store 31. Skipping.
Insufficient data for Trial Store 88 or Control Store 206. Skipping.


In [16]:
def alternative_metrics(trial_data, control_data, trial_period):
    """
    Compute alternative metrics to compare trial and control stores.

    Args:
        trial_data (pd.DataFrame): Data for the trial store.
        control_data (pd.DataFrame): Data for the control store.
        trial_period (tuple): Start and end dates of the trial period (YYYY-MM-DD).

    Returns:
        dict: Metrics comparing trial and control stores.
    """
    metrics = {}
    
    # Filter data for trial period
    trial_period_data = trial_data[
        (trial_data['DATE'] >= trial_period[0]) & (trial_data['DATE'] <= trial_period[1])
    ]
    control_period_data = control_data[
        (control_data['DATE'] >= trial_period[0]) & (control_data['DATE'] <= trial_period[1])
    ]
    
    # Compute total sales
    trial_sales = trial_period_data['total_sales'].sum()
    control_sales = control_period_data['total_sales'].sum()
    
    # Compute average transactions per customer
    trial_transactions_per_customer = (
        trial_period_data['transactions_per_customer'].mean()
    )
    control_transactions_per_customer = (
        control_period_data['transactions_per_customer'].mean()
    )
    
    # Compute percentage changes
    metrics['sales_percentage_change'] = (
        (trial_sales - control_sales) / control_sales * 100 if control_sales != 0 else None
    )
    metrics['transactions_percentage_change'] = (
        (trial_transactions_per_customer - control_transactions_per_customer) /
        control_transactions_per_customer * 100 if control_transactions_per_customer != 0 else None
    )
    
    # Compute absolute and normalized differences
    metrics['sales_absolute_difference'] = trial_sales - control_sales
    metrics['transactions_absolute_difference'] = (
        trial_transactions_per_customer - control_transactions_per_customer
    )
    metrics['sales_normalized_difference'] = (
        abs(trial_sales - control_sales) / control_sales if control_sales != 0 else None
    )
    metrics['transactions_normalized_difference'] = (
        abs(trial_transactions_per_customer - control_transactions_per_customer) /
        control_transactions_per_customer if control_transactions_per_customer != 0 else None
    )
    
    return metrics


In [18]:
monthly_metrics['transactions_per_customer'] = (
    monthly_metrics['total_transactions'] / monthly_metrics['total_customers']
)

KeyError: 'total_transactions'

In [17]:
# Define the trial period
trial_period = ('2019-01-01', '2019-03-31')

# Compare each trial and control store
for trial_store_num, control_store_info in best_control_stores.items():
    control_store_num = control_store_info[0]
    
    # Get trial and control data
    trial_store_data = monthly_metrics[monthly_metrics['STORE_NBR'] == trial_store_num]
    control_store_data = monthly_metrics[monthly_metrics['STORE_NBR'] == control_store_num]
    
    # Compute alternative metrics
    results = alternative_metrics(trial_store_data, control_store_data, trial_period)
    
    print(f"Results for Trial Store {trial_store_num} vs Control Store {control_store_num}:")
    print(results)


KeyError: 'transactions_per_customer'